In [1]:
%pylab inline

import os, pres_style

Populating the interactive namespace from numpy and matplotlib


In [2]:
path = os.path.expandvars("$master/inputs/tta_cb13/ssag")

colors = genfromtxt("{}/SSAG_1.00TauV.log".format(path), dtype=None, names=True, skip_header=2)
params = genfromtxt("{}/true_params.log".format(path), dtype=None, names=True)

In [3]:
#filter nd galaxies in params
mask = array([not name.endswith("nd") for name in params["Galaxy"]], dtype=bool)
params_clean = params[mask]

#find the union between colors and params
mask = array([name in params_clean["Galaxy"] for name in colors["Seq"]], dtype=bool)
colors_clean = colors[mask]

In [4]:
#write joint tables
c_fields = "u g r i z".split()
p_fields = "Masszobs rFWLAyr ZZsun Av_eff".split()
t_fields = "name u g r i z M_lib log_t_L_lib log_Z_L_lib Av_lib".split()
t_dtypes = zip(t_fields, ("S18 "+9*"<f8 ").split())

p_map = dict(zip(p_fields, t_fields[6:]))

catalog = zeros(params_clean.size, dtype=t_dtypes)

catalog["name"] = params_clean["Galaxy"]
for field in c_fields: catalog[field] = colors_clean[field]
for p_field, t_field in p_map.iteritems(): catalog[t_field] = params_clean[p_field]

fmt = "%16s"+5*"%8.4f"+4*"%12.5f"
hdr = ("%14s"+5*"%8s"+4*"%12s")%tuple(t_fields)
savetxt("{}/ssag_cb13_catalog.txt".format(path), catalog, fmt=fmt, header=hdr)